In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm

from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [3]:
X_train = pd.read_csv("data/train_inter.csv")
X_test = pd.read_csv("data/test_inter.csv")


In [4]:
q_train = pd.read_csv("data/freq_train.csv")
q_test = pd.read_csv("data/freq_test.csv")

In [5]:
X_train = pd.concat([X_train, q_train], axis=1)
X_test = pd.concat([X_test, q_test], axis=1)

In [7]:
X_train

,id,qid1,qid2,question1,question2,word_match,tfidf_word_match,q1,q2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0.727273,0.772164,0.000001,0.000002
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0.307692,0.361758,0.000010,0.000004
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0.363636,0.355191,0.000002,0.000001
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0.000000,0.000000,0.000001,0.000001
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0.000000,0.000000,0.000004,0.000001
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",0.470588,0.510771,0.000001,0.000001
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0.000000,0.000000,0.000001,0.000001
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,0.500000,0.645836,0.000001,0.000001
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0.500000,0.396755,0.000002,0.000004
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0.363636,0.503203,0.000001,0.000001


In [6]:
X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))
X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())
X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [7]:
X = map(lambda x: ' '.join(x), X_train['question1'])
X1 = map(lambda x: ' '.join(x), X_train['question2'])
Y = map(lambda x: ' '.join(x), X_test['question1'])
Y1 = map(lambda x: ' '.join(x), X_test['question2'])

hw = HashingVectorizer(n_features=200).fit(X_train['question1'] + X_train['question2'])


In [8]:
X = pd.DataFrame(hw.transform(X).todense())
X1 = pd.DataFrame(hw.transform(X1).todense())
Y = pd.DataFrame(hw.transform(Y).todense())
Y1 = pd.DataFrame(hw.transform(Y1).todense())

In [9]:
tfidf = feature_extraction.text.TfidfTransformer()
tfidf.fit(X+X1)

X = pd.DataFrame(tfidf.transform(X).todense())
X1 = pd.DataFrame(tfidf.transform(X1).todense())
Y = pd.DataFrame(tfidf.transform(Y).todense())
Y1 = pd.DataFrame(tfidf.transform(Y1).todense())

In [10]:
X_train = pd.concat([X_train, X], axis=1, join_axes=[X_train.index])
del X
X_train = pd.concat([X_train, X1], axis=1, join_axes=[X_train.index])
del X1
X_test = pd.concat([X_test, Y], axis=1, join_axes=[X_test.index])
del Y
X_test = pd.concat([X_test, Y1], axis=1, join_axes=[X_test.index])
del Y1

In [13]:
X_train[:2]

,id,qid1,qid2,question1,question2,word_match,tfidf_word_match,q1,q2,0,...,190,191,192,193,194,195,196,197,198,199
0,0,1,2,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...",0.727273,0.772164,0.000001,0.000002,0.0,...,0.0,0.0,0.325385,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,3,4,"[what, is, the, story, of, kohinoor, (koh-i-no...","[what, would, happen, if, the, indian, governm...",0.307692,0.361758,0.000010,0.000004,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../tmp/GoogleNews-vectors-negative300.bin', binary=True)

In [29]:
def to_vec(x):
    result = word2vec.word_vec('yes') - word2vec.word_vec('yes')
    co = 0
    for i in x:
        if i in word2vec.vocab:
            result += word2vec.word_vec(i)
            co += 1
    return result / co

In [30]:
def norm1(x, y):
    arr = []
    for i in x.index:
        arr += [LA.norm(x[i] - y[i])]
    return arr

In [31]:
tmp1 = X_train['question1'].apply(lambda x: to_vec(x))
tmp2 = X_train['question2'].apply(lambda x: to_vec(x))
from numpy import linalg as LA
n1 = norm1(tmp1, tmp2)
del tmp1
del tmp2

In [11]:
del X_train['question1']
del X_train['question2']
del X_train['qid1']
del X_train['qid2']
del X_train['id']

del X_test['question1']
del X_test['question2']
del X_test['test_id']

In [15]:
X_train

,word_match,tfidf_word_match,q1,q2,0,1,2,3,4,5,...,190,191,192,193,194,195,196,197,198,199
0,0.727273,0.772164,0.000001,0.000002,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.325385,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
1,0.307692,0.361758,0.000010,0.000004,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
2,0.363636,0.355191,0.000002,0.000001,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.000001,0.000001,0.000000,0.000000,0.000000,0.000000,0.0,-0.219821,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.000004,0.000001,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
5,0.470588,0.510771,0.000001,0.000001,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,-0.202691,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
6,0.000000,0.000000,0.000001,0.000001,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
7,0.500000,0.645836,0.000001,0.000001,0.000000,0.000000,0.000000,0.000000,0.0,-0.301252,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
8,0.500000,0.396755,0.000002,0.000004,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.266967,0.000000,0.0,0.0
9,0.363636,0.503203,0.000001,0.000001,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0


In [12]:
#tmp = abs(X_train.ix[:, 2:202] * X_train.ix[:, -200:])
#X_train = pd.concat([X_train, tmp], axis=1, join_axes=[X_train.index])
tmp = abs(X_train.ix[:, 4:204] - X_train.ix[:, -200:])
X_train = pd.concat([X_train, tmp], axis=1, join_axes=[X_train.index])

#tmp = abs(X_test.ix[:, 2:202] * X_test.ix[:, -200:])
#X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])
tmp = abs(X_test.ix[:, 4:204] - X_test.ix[:, -200:])
X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])

In [34]:
X_train['n1'] = n1

In [19]:
X_train[:5]

,word_match,tfidf_word_match,q1,q2,0,1,2,3,4,5,...,190,191,192,193,194,195,196,197,198,199
0,0.727273,0.772164,1.818125e-07,3.636250e-07,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.016757,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.307692,0.361758,1.454500e-06,5.454375e-07,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.209907,0.0,0.0,0.0
2,0.363636,0.355191,3.636250e-07,1.818125e-07,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.186087,0.0,0.0,0.0
3,0.000000,0.000000,1.818125e-07,1.818125e-07,0.0,0.0,0.0,0.0,0.0,-0.219821,...,0.489537,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.000000,0.000000,5.454375e-07,1.818125e-07,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [18]:
X_test[:5]

,word_match,tfidf_word_match,q1,q2,0,1,2,3,4,5,...,190,191,192,193,194,195,196,197,198,199
0,0.266667,0.274019,1.818125e-07,1.818125e-07,0.0,0.0,0.000000,0.0,0.0,-0.156312,...,0.0,0.228360,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0
1,0.500000,0.480962,3.636250e-07,3.636250e-07,0.0,0.0,0.000000,0.0,0.0,-0.164904,...,0.0,0.295051,0.0,0.0,0.0,0.000000,0.0,0.30673,0.0,0.0
2,0.444444,0.468893,1.818125e-07,1.818125e-07,0.0,0.0,-0.316774,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0
3,0.000000,0.000000,1.818125e-07,1.818125e-07,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000,0.0,0.0
4,0.800000,1.000000,1.818125e-07,1.818125e-07,0.0,0.0,0.000000,0.0,0.0,-0.252158,...,0.0,0.000000,0.0,0.0,0.0,0.001533,0.0,0.00000,0.0,0.0


In [16]:
print(X_test.q1.mean())
print(X_train.q1.mean())


9.668771881958455e-07
9.009205147102744e-07


In [ ]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 1100

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 50,
    #'max_depth': 6
    #'num_class': 3
    
}

pred_train = np.zeros(len(y_train))
xgbs = []
sc,sc_mean = [],[]

for itr, ite in folds:
    ypred = []
    for i in range(1):
               
        param['seed'] = i+1
        
        #xgboost
        
        Xdatatrain = xgb.DMatrix(data=X_train.ix[itr, :].values,
                                     label=y_train.ix[itr].values)
        Xdataval = xgb.DMatrix(data=X_train.ix[ite, :].values,
                                     label=y_train.ix[ite].values)

        plst = list(param.items())
        watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
        bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)
        
        
        #rc = ensemble.ExtraTreesClassifier(n_estimators=1300, criterion='gini', max_depth=None, n_jobs=-1)
        #pred_train[ite] = bst.predict(Xdataval)
        #neigh = KNeighborsClassifier(n_neighbors=2)
        #neigh.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = bst.predict(Xdataval)
        #ypred = neigh.predict_proba(X_train.ix[ite, :])
        xgbs.append(bst)
        
        '''
        # train
        lgb = lgbm.sklearn.LGBMClassifier(n_estimators=300, seed=0, **params)
        lgb.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = lgb.predict_proba(X_train.ix[ite, :])[:, 1]

        
        model.fit(X_train.ix[itr, :].values, y_train_cat[itr],
            epochs=60,
            batch_size=1000)
        ypred = model.predict(X_train.ix[ite, :].values)
        '''
    #ypred = sum(ypred) / len(ypred) 0.401408 0.392476
    pred_train[ite] = ypred
    
    
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

[0]	train-logloss:0.645048	eval-logloss:0.645487
[15]	train-logloss:0.393249	eval-logloss:0.397601
[30]	train-logloss:0.349433	eval-logloss:0.356234
[45]	train-logloss:0.336142	eval-logloss:0.344962
[60]	train-logloss:0.327796	eval-logloss:0.339054
[75]	train-logloss:0.322381	eval-logloss:0.335725
[90]	train-logloss:0.3176	eval-logloss:0.33283
[105]	train-logloss:0.313369	eval-logloss:0.330551
[120]	train-logloss:0.309689	eval-logloss:0.328676
[135]	train-logloss:0.306086	eval-logloss:0.326806
[150]	train-logloss:0.30297	eval-logloss:0.325261
[165]	train-logloss:0.299935	eval-logloss:0.323771
[180]	train-logloss:0.297302	eval-logloss:0.322677
[195]	train-logloss:0.295008	eval-logloss:0.32171
[210]	train-logloss:0.292631	eval-logloss:0.32072
[225]	train-logloss:0.290523	eval-logloss:0.319923
[240]	train-logloss:0.288762	eval-logloss:0.319196
[255]	train-logloss:0.28667	eval-logloss:0.318362
[270]	train-logloss:0.284769	eval-logloss:0.317644
[285]	train-logloss:0.283044	eval-logloss:0.31

In [13]:
def foo(x):
    a = 0.165 / 0.37
    b = (1 - 0.165) / (1 - 0.37) 
    return  a * x / (a * x + b * (1 - x))

In [ ]:
pred_train = foo(pred_train)

In [ ]:
pred_train = pd.DataFrame(pred_train)
pred_train.columns = ['y']
pred_train.to_csv("stacking/xgb_10.csv", index=None)

In [ ]:
1+1

In [ ]:
1

### Тест

In [14]:
submit = pd.read_csv("data/sample_submission.csv")

In [18]:
lgb = lgbm.sklearn.LGBMClassifier(n_estimators=300, seed=0, **params)
lgb.fit(X_train, y_train)
test_pred = lgb.predict_proba(X_test)[:, 1]

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 1100

In [ ]:
Xdatatrain = xgb.DMatrix(data=X_train.ix[:, :].values,
                                     label=y_train.ix[:].values)
Xdataval = xgb.DMatrix(data=X_train.values,
                                     label=y_train.values)
Xdatatest = xgb.DMatrix(data=X_test.values)


plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
#bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


#test_pred = bst.predict(Xdatatest)



In [16]:
Xdatatrain = xgb.DMatrix(data=X_train.ix[:, :].values,
                                     label=y_train.ix[:].values)

In [17]:
Xdataval = xgb.DMatrix(data=X_train.ix[:10, :].values,
                                     label=y_train.ix[:10].values)

In [20]:
Xdatatest = xgb.DMatrix(data=X_test.ix[:, :].values)


In [21]:

plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]

In [22]:
bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


[0]	train-logloss:0.645145	eval-logloss:0.653016
[15]	train-logloss:0.394068	eval-logloss:0.437614
[30]	train-logloss:0.350926	eval-logloss:0.400341
[45]	train-logloss:0.337777	eval-logloss:0.401014
[60]	train-logloss:0.330209	eval-logloss:0.387795
[75]	train-logloss:0.324804	eval-logloss:0.387937
[90]	train-logloss:0.320158	eval-logloss:0.383231
[105]	train-logloss:0.316191	eval-logloss:0.384437
[120]	train-logloss:0.313145	eval-logloss:0.380075
[135]	train-logloss:0.309691	eval-logloss:0.37662
[150]	train-logloss:0.306753	eval-logloss:0.375377
[165]	train-logloss:0.304356	eval-logloss:0.370649
[180]	train-logloss:0.301772	eval-logloss:0.358448
[195]	train-logloss:0.299199	eval-logloss:0.355894
[210]	train-logloss:0.296768	eval-logloss:0.359679
[225]	train-logloss:0.294945	eval-logloss:0.36081
[240]	train-logloss:0.293169	eval-logloss:0.354961
[255]	train-logloss:0.291001	eval-logloss:0.353774
[270]	train-logloss:0.289022	eval-logloss:0.350644
[285]	train-logloss:0.28755	eval-logloss:

In [ ]:
test_pred = bst.predict(Xdatatest)


In [ ]:
test_pred = foo(test_pred)

In [ ]:
submit['is_duplicate'] = test_pred

In [26]:
submit.to_csv("stacking/xgb_10_test.csv", index=None)

In [27]:
submit

,test_id,is_duplicate
0,0,0.014965
1,1,0.335637
2,2,0.182598
3,3,0.000644
4,4,0.353774
5,5,0.024499
6,6,0.972693
7,7,0.338457
8,8,0.450769
9,9,0.007340
